# Structured Outputs

LLMs regurgitate out text and that is great for so many applications. But in order to build strong, robust systems and applications, we need to make sense of the chaos sometimes by receiving a pre-determined structured output everytime an LLM is called.

## As always, libraries first!

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# check if API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key")
if not GEMINI_API_KEY:
    raise ValueError("Missing Gemini API key")
if not ANTHROPIC_API_KEY:
    raise ValueError("Missing Anthropic API key")

ValueError: Missing Gemini API key

## The Workflow

```mermaid
graph LR
    A[Generate Ticket] --> B[Respond to Ticket]
    B --> C[Evaluate Response]
    C --> B
    C --> D[Final Output]
```

## Introducing the Pydantic library

In [ ]:
# classes
from pydantic import BaseModel

class CustomerTicket(BaseModel):
    ticket: str
    priority: str
    assigned_to: str

class TicketResponse(BaseModel):
    response: str
    resolution_time: str

class TicketEvaluation(BaseModel):
    passed: bool
    feedback: str

## Calling OpenAI to generate support tickets

In [3]:
# client
client = OpenAI()

In [4]:
# messages list
user_message = "I want you to generate a customer support ticket for a 3rd party tech re-seller. "
user_message += "The ticket should be a single sentence describing a common issue a customer might face with their product or service. "
user_message += "Please ensure the ticket is varied and covers different types of problems."

messages = [{"role": "user", "content": user_message}]

In [5]:
# normal response
response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)

normal_response = response.choices[0].message.content
display(Markdown(f"### Normal Response:\n{normal_response}"))

### Normal Response:
The customer's account login is repeatedly failing despite entering the correct credentials.

In [6]:
# structured response, will generate a "random" ticket that fits the CustomerTicket schema
structured_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    # new, specify the response format
    response_format=CustomerTicket
)
# new, access message.parsed instead of message.content
structured_response = structured_response.choices[0].message.parsed
display(Markdown(f"### Structured Response:\n{structured_response}"))

### Structured Response:
ticket='The customer reports that the software subscription is not activating despite multiple attempts, and they are unable to access their purchased features.' priority='High' assigned_to='Technical Support Team'

In [7]:
structured_response.ticket

'The customer reports that the software subscription is not activating despite multiple attempts, and they are unable to access their purchased features.'

In [8]:
structured_response.priority

'High'

## Responding to the ticket

In [9]:
# messages list
message = "You are to propose a resolution for the following customer support ticket. \n\n"
message += f"Ticket: {structured_response.ticket}\n"
message += f"Priority: {structured_response.priority}\n\n"

messages = [{"role": "user", "content": message}]

In [10]:
# structured response
ticket_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=TicketResponse
)

ticket_response = ticket_response.choices[0].message.parsed
display(Markdown(f"### Response:\n{ticket_response.response}"))
display(Markdown(f"### Resolution Time:\n{ticket_response.resolution_time}"))

### Response:
Dear Customer,

Thank you for reaching out. We understand the urgency of your issue and apologize for the inconvenience caused. To resolve the activation problem, please try the following steps:

1. Ensure that your internet connection is stable.
2. Restart your device and attempt to activate the software again.
3. If you are behind a firewall or using security software, please verify that it isn't blocking the activation process.
4. Clear your browser's cache and cookies if activating via web.
5. Make sure your subscription details are correct and up to date.

If the problem persists after trying these steps, please provide us with your account details and any error messages received. We will then investigate further and assist you in activating your subscription promptly.

Thank you for your patience.

Best regards,
Customer Support Team

### Resolution Time:
Within 24 hours

## Lets evaluate our response

In [14]:
# messages list
message = "You are to evaluate the proposed resolution for the following customer support ticket. "
message += "You will determine if the proposed resolution is appropriate for the ticket and priority level. "
message += "tickets\n\n"
message += f"Ticket: {structured_response.ticket}\n"
message += f"Priority: {structured_response.priority}\n\n"
message += f"Proposed Resolution: {ticket_response.response}\n"
message += f"Proposed Resolution {ticket_response.resolution_time}\n\n"

messages = [{"role": "user", "content": message}]

In [15]:
messages

[{'role': 'user',
  'content': "You are to evaluate the proposed resolution for the following customer support ticket. You will determine if the proposed resolution is appropriate for the ticket and priority level. tickets\n\nTicket: The customer reports that the software subscription is not activating despite multiple attempts, and they are unable to access their purchased features.\nPriority: High\n\nProposed Resolution: Dear Customer,\n\nThank you for reaching out. We understand the urgency of your issue and apologize for the inconvenience caused. To resolve the activation problem, please try the following steps:\n\n1. Ensure that your internet connection is stable.\n2. Restart your device and attempt to activate the software again.\n3. If you are behind a firewall or using security software, please verify that it isn't blocking the activation process.\n4. Clear your browser's cache and cookies if activating via web.\n5. Make sure your subscription details are correct and up to date

In [16]:
# evaluate response
evaluator_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=TicketEvaluation
)

evaluator_response = evaluator_response.choices[0].message.parsed
display(Markdown(f"### Passed:\n{evaluator_response.passed}"))
display(Markdown(f"### Feedback:\n{evaluator_response.feedback}"))

### Passed:
True

### Feedback:
The proposed resolution appropriately addresses the customer's activation issue with detailed troubleshooting steps suitable for a high-priority case. It acknowledges the urgency, offers clear instructions, and provides an escalation pathway if the problem persists. Overall, it is an effective and suitable response for the ticket.

<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
  <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
    <li>Hey everyone! Ready to flex those agentic muscles? 🎉 Build a workflow just like the ticket system above, but for <b>product reviews</b>!</li>
    <li>Your workflow should:
      <ul>
        <li>Generate a product review (think: electronics, books, or your favorite kitchen gadget)</li>
        <li>Respond to the review (company reply, moderation, or a witty bot response)</li>
        <li>Evaluate the response (is it helpful, polite, and on point?)</li>
      </ul>
    </li>
    <li>Use structured outputs and Pydantic models for each step, just like we did above.</li>
    <li>Include an evaluator step to assess the quality of the response.</li>
    <li>Here’s a suggested workflow to get your creative gears turning:</li>
  </ul>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>
</div>

### Suggested Workflow

```mermaid
graph LR
    A[Generate Review] --> B[Respond to Review]
    B --> C[Evaluate Response]
    C --> B
    C --> D[Final Output]
```

Try to use structured outputs and Pydantic models for each step, just like in the notebook above. Include an evaluator step to assess the quality of the response.